Convolutional Neural Network 

In [3]:
import tensorflow as tf 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 



Part 1 - Data Preprocessing 

In [4]:
# Data augmentation pipeline (applied to image tensors)
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.2),
])

# Helper to rescale pixel values to [0,1]
rescale = tf.keras.layers.Rescaling(1./255)

# Load dataset from directory using the proper utility
dataset_dir = r"C:\Users\ladom\Desktop\LLm\data\dataset\training_set"
training_set = tf.keras.utils.image_dataset_from_directory(
    dataset_dir,
    labels='inferred',
    label_mode='binary',
    batch_size=32,
    image_size=(150, 150),
    shuffle=True,
    seed=42
)

# Apply rescaling and augmentation to the dataset
training_set = training_set.map(lambda x, y: (rescale(x), y))
training_set = training_set.map(lambda x, y: (data_augmentation(x, training=True), y))

# Prefetch for performance
training_set = training_set.prefetch(buffer_size=tf.data.AUTOTUNE)

print("Training dataset prepared:", training_set)

Found 6935 files belonging to 2 classes.
Training dataset prepared: <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 150, 150, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>


preprocessing the test set 

In [5]:
# Preprocess the test set (do NOT apply augmentation here)
testset_dir = r"C:\Users\ladom\Desktop\LLm\data\dataset\test_set"

# Create test dataset (no augmentation, no shuffling)
test_set = tf.keras.utils.image_dataset_from_directory(
    testset_dir,
    labels='inferred',
    label_mode='binary',    # match training label_mode
    batch_size=32,
    image_size=(150, 150),
    shuffle=False
)

# Apply the SAME rescaling used for training (no augmentation)
test_set = test_set.map(lambda x, y: (rescale(x), y))

# Prefetch for performance
test_set = test_set.prefetch(buffer_size=tf.data.AUTOTUNE)

print("Test dataset prepared:", test_set)

Found 2000 files belonging to 2 classes.
Test dataset prepared: <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 150, 150, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>


# Part 2 - Building the CNN
step 1 Convolution 
step 2 pooling 
adding second layer
step 3 Flattening
step 4 Full Connection

In [17]:
cnn = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, kernel_size=3, activation='relu', input_shape=[150, 150, 3]),
    tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
    tf.keras.layers.Conv2D(32, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(units=1, activation='sigmoid')  # For one-hot encoded binary labels
])

# Part 3 Training the CNN

In [18]:
# Compiling the CNN
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [20]:
# Training the CNN on the Training and evaluating it on the Test set 
cnn.fit(x=training_set, validation_data=test_set, epochs=25)

Epoch 1/25
217/217 ━━━━━━━━━━━━━━━━━━━━ 28s 128ms/step - accuracy: 0.5882 - loss: 0.6780 - val_accuracy: 0.5730 - val_loss: 0.6713
Epoch 2/25
217/217 ━━━━━━━━━━━━━━━━━━━━ 23s 106ms/step - accuracy: 0.6362 - loss: 0.6467 - val_accuracy: 0.6385 - val_loss: 0.6465
Epoch 3/25
217/217 ━━━━━━━━━━━━━━━━━━━━ 29s 134ms/step - accuracy: 0.6567 - loss: 0.6289 - val_accuracy: 0.6765 - val_loss: 0.6225
Epoch 4/25
217/217 ━━━━━━━━━━━━━━━━━━━━ 30s 138ms/step - accuracy: 0.6738 - loss: 0.6032 - val_accuracy: 0.6985 - val_loss: 0.5867
Epoch 5/25
217/217 ━━━━━━━━━━━━━━━━━━━━ 25s 116ms/step - accuracy: 0.6986 - loss: 0.5787 - val_accuracy: 0.6980 - val_loss: 0.5851
Epoch 6/25
217/217 ━━━━━━━━━━━━━━━━━━━━ 26s 121ms/step - accuracy: 0.7185 - loss: 0.5605 - val_accuracy: 0.7270 - val_loss: 0.5484
Epoch 7/25
217/217 ━━━━━━━━━━━━━━━━━━━━ 30s 137ms/step - accuracy: 0.7246 - loss: 0.5423 - val_accuracy: 0.7430 - val_loss: 0.5376
Epoch 8/25
217/217 ━━━━━━━━━━━━━━━━━━━━ 29s 134ms/step - accuracy: 0.7377 - loss: 0

# Part 4 Making a single prediction 

In [22]:
from keras.preprocessing import image 
test_image=image.load_img(r'C:\Users\ladom\Desktop\LLm\data\dataset\single_prediction\cat_or_dog_1.jpg', target_size=(150,150))
test_image=image.img_to_array(test_image) 
test_image=np.expand_dims(test_image, axis=0) 
result=cnn.predict(test_image) 
if result[0][0]>0.5:
    prediction="dog"
else:
    prediction="cat" 
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
dog
